In [ ]:
from __future__ import annotations

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np

import qubex.pulse as qp
from qubex.simulator import (
    Control,
    Coupling,
    QuantumSimulator,
    QuantumSystem,
    Transmon,
)

In [ ]:
# units: ns, GHz
transmon_dimension = 3
control_frequency = 7.648
target_frequency = 8.275
anharmonicity = -0.333
relaxation_rate = 5e-5
dephasing_rate = 5e-5
coupling_strength = 0.01

qubits = [
    Transmon(
        label="Q01",
        dimension=transmon_dimension,
        frequency=control_frequency,
        anharmonicity=anharmonicity,
        relaxation_rate=relaxation_rate,
        dephasing_rate=dephasing_rate,
    ),
    Transmon(
        label="Q02",
        dimension=transmon_dimension,
        frequency=target_frequency,
        anharmonicity=anharmonicity,
        relaxation_rate=relaxation_rate,
        dephasing_rate=dephasing_rate,
    ),
]

couplings = [
    Coupling(
        pair=("Q01", "Q02"),
        strength=coupling_strength,
    ),
]

system = QuantumSystem(
    objects=qubits,
    couplings=couplings,
)

simulator = QuantumSimulator(system)

In [ ]:
target_qubit = qubits[0]

pulse_duration = 30  # ns
ramp_duration = 10  # ns

In [ ]:
hpi_pulse = qp.FlatTop(
    duration=pulse_duration,
    amplitude=0.5 * np.pi / (pulse_duration - ramp_duration),
    tau=ramp_duration,
)
hpi_pulse.plot(divide_by_two_pi=True)

pi_pulse = hpi_pulse.repeated(2)
pi_pulse.plot(divide_by_two_pi=True)

In [ ]:
result = simulator.mesolve(
    controls=[
        Control(
            target=target_qubit.label,
            frequency=target_qubit.frequency,
            waveform=hpi_pulse,
        )
    ],
    initial_state=simulator.system.ground_state,
)

result.show_last_population(target_qubit.label)
result.plot_population_dynamics(target_qubit.label)
result.plot_bloch_vectors(target_qubit.label)
result.display_bloch_sphere(target_qubit.label)

In [ ]:
result = simulator.mesolve(
    controls=[
        Control(
            target=target_qubit.label,
            frequency=target_qubit.frequency,
            waveform=pi_pulse,
        )
    ],
    initial_state=simulator.system.ground_state,
)

result.show_last_population(target_qubit.label)
result.plot_population_dynamics(target_qubit.label)
result.plot_bloch_vectors(target_qubit.label)
result.display_bloch_sphere(target_qubit.label)